In [ ]:
# import os, random, numpy as np
# import torch
# from torch.utils.data import DataLoader
# import torchvision.transforms.functional as TF
# from torchvision.datasets import VOCDetection, VOCSegmentation

# VOC07_TRAIN_ROOT = "/kaggle/input/pascal-voc-2007/VOCtrainval_06-Nov-2007"
# VOC07_TEST_ROOT  = "/kaggle/input/pascal-voc-2007/VOCtest_06-Nov-2007"

# VOC_CLASSES = [
#     "aeroplane","bicycle","bird","boat","bottle",
#     "bus","car","cat","chair","cow",
#     "diningtable","dog","horse","motorbike","person",
#     "pottedplant","sheep","sofa","train","tvmonitor"
# ]

# CLASS_TO_IDX = {cls: i + 1 for i, cls in enumerate(VOC_CLASSES)}  


# def parse_voc_annotation(target):
#     objects = target["annotation"].get("object", [])
#     if isinstance(objects, dict):
#         objects = [objects]

#     boxes, labels = [], []

#     for obj in objects:
#         name = obj["name"]
#         if name not in CLASS_TO_IDX:
#             continue

#         bbox = obj["bndbox"]
#         xmin = float(bbox["xmin"]) - 1
#         ymin = float(bbox["ymin"]) - 1
#         xmax = float(bbox["xmax"]) - 1
#         ymax = float(bbox["ymax"]) - 1

#         boxes.append([xmin, ymin, xmax, ymax])
#         labels.append(CLASS_TO_IDX[name])

#     if len(boxes) == 0:
#         boxes = torch.zeros((0, 4), dtype=torch.float32)
#         labels = torch.zeros((0,), dtype=torch.int64)
#     else:
#         boxes = torch.tensor(boxes, dtype=torch.float32)
#         labels = torch.tensor(labels, dtype=torch.int64)

#     return {"boxes": boxes, "labels": labels}


# MEAN = [0.485, 0.456, 0.406]
# STD  = [0.229, 0.224, 0.225]


# def transform_train_detection(img, target):
#     img = TF.to_tensor(img)
#     img = TF.normalize(img, MEAN, STD)
#     target = parse_voc_annotation(target)

#     if random.random() < 0.5:
#         img = TF.hflip(img)
#         _, H, W = img.shape
#         if target["boxes"].shape[0] > 0:
#             xmin, ymin, xmax, ymax = target["boxes"].unbind(1)
#             target["boxes"] = torch.stack(
#                 [W - xmax - 1, ymin, W - xmin - 1, ymax], dim=1
#             )
#     return img, target


# def transform_val_detection(img, target):
#     img = TF.to_tensor(img)
#     img = TF.normalize(img, MEAN, STD)
#     target = parse_voc_annotation(target)
#     return img, target


# def transform_train_segmentation(img, mask):
#     if random.random() < 0.5:
#         img = TF.hflip(img)
#         mask = TF.hflip(mask)

#     img = TF.to_tensor(img)
#     img = TF.normalize(img, MEAN, STD)
#     mask = torch.as_tensor(np.array(mask), dtype=torch.int64)
#     return img, mask


# def transform_val_segmentation(img, mask):
#     img = TF.to_tensor(img)
#     img = TF.normalize(img, MEAN, STD)
#     mask = torch.as_tensor(np.array(mask), dtype=torch.int64)
#     return img, mask


# train_dataset_det = VOCDetection(
#     root=VOC07_TRAIN_ROOT,
#     year="2007",
#     image_set="trainval",
#     download=False,
#     transforms=transform_train_detection
# )

# test_dataset_det = VOCDetection(
#     root=VOC07_TEST_ROOT,
#     year="2007",
#     image_set="test",
#     download=False,
#     transforms=transform_val_detection
# )


# train_dataset_seg = VOCSegmentation(
#     root=VOC07_TRAIN_ROOT,
#     year="2007",
#     image_set="trainval",
#     download=False,
#     transforms=transform_train_segmentation
# )

# val_dataset_seg = VOCSegmentation(
#     root=VOC07_TEST_ROOT,
#     year="2007",
#     image_set="test",
#     download=False,
#     transforms=transform_val_segmentation
# )


# def collate_fn_det(batch):
#     images = [b[0] for b in batch]
#     targets = [b[1] for b in batch]
#     return images, targets


# def collate_fn_seg(batch):
#     maxH = max(img.shape[1] for img, _ in batch)
#     maxW = max(img.shape[2] for img, _ in batch)

#     images, masks = [], []

#     for img, mask in batch:
#         C, H, W = img.shape
#         padded_img = torch.zeros((C, maxH, maxW))
#         padded_img[:, :H, :W] = img

#         padded_mask = torch.full((maxH, maxW), 255, dtype=torch.long)
#         padded_mask[:H, :W] = mask

#         images.append(padded_img)
#         masks.append(padded_mask)

#     return torch.stack(images), torch.stack(masks)


# train_loader_det = DataLoader(
#     train_dataset_det,
#     batch_size=4,
#     shuffle=True,
#     num_workers=2,
#     collate_fn=collate_fn_det
# )

# test_loader_det = DataLoader(
#     test_dataset_det,
#     batch_size=1,
#     shuffle=False,
#     num_workers=2,
#     collate_fn=collate_fn_det
# )

# train_loader_seg = DataLoader(
#     train_dataset_seg,
#     batch_size=4,
#     shuffle=True,
#     num_workers=2,
#     collate_fn=collate_fn_seg
# )

# val_loader_seg = DataLoader(
#     val_dataset_seg,
#     batch_size=4,
#     shuffle=False,
#     num_workers=2,
#     collate_fn=collate_fn_seg
# )


# print("Detection:", len(train_dataset_det), len(test_dataset_det))
# print("Segmentation:", len(train_dataset_seg), len(val_dataset_seg))

# imgs, targets = next(iter(train_loader_det))
# print(imgs[0].shape, targets[0].keys())

# imgs, masks = next(iter(train_loader_seg))
# print(imgs.shape, masks.shape)


In [ ]:
# import torchvision
# from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
# from torchvision.models.detection import FasterRCNN
# from torchvision.models.segmentation import deeplabv3_resnet50

# moco_checkpoint = torch.load("/kaggle/input/pretrained-weights-mocov2/moco_v2_800ep_pretrain.pth.tar", map_location="cpu")
# state_dict = moco_checkpoint["state_dict"]
# new_state_dict = {}
# for k, v in state_dict.items():
#     if k.startswith("module.encoder_q."):
#         new_key = k[len("module.encoder_q."):]
#         new_state_dict[new_key] = v
#     elif k.startswith("encoder_q."):
#         new_key = k[len("encoder_q."):]
#         new_state_dict[new_key] = v

# backbone_supervised = resnet_fpn_backbone("resnet50", pretrained=True, trainable_layers=5)
# model_det_baseline = FasterRCNN(backbone_supervised, num_classes=21)

# backbone_moco = resnet_fpn_backbone("resnet50", pretrained=False, trainable_layers=5)
# backbone_moco.body.load_state_dict(new_state_dict, strict=False)
# model_det_moco = FasterRCNN(backbone_moco, num_classes=21)

# model_seg_baseline = deeplabv3_resnet50(pretrained=False, pretrained_backbone=True, num_classes=21, aux_loss=True)

# model_seg_moco = deeplabv3_resnet50(pretrained=False, pretrained_backbone=False, num_classes=21, aux_loss=True)
# model_seg_moco.backbone.body.load_state_dict(new_state_dict, strict=False)


In [ ]:
# import torch.nn.functional as F

# def evaluate_detection(model, data_loader):
#     model.eval()
#     preds_by_class = {cls: [] for cls in range(1, 21)}  
#     gt_by_class = {cls: {} for cls in range(1, 21)}     
#     npos = {cls: 0 for cls in range(1, 21)}             

#     with torch.no_grad():
#         for img_idx, (images, targets) in enumerate(data_loader):
#             images = [img.to(device) for img in images]
#             outputs = model(images)
#             for i, output in enumerate(outputs):
#                 idx = img_idx * data_loader.batch_size + i 
#                 for t in targets[i]["labels"].tolist():
#                     if t == 0: 
#                         continue
#                     gt_boxes = targets[i]["boxes"].tolist()
#                     gt_by_class[t].setdefault(idx, []).append(gt_boxes[targets[i]["labels"].tolist().index(t)])
#                     npos[t] += 1
#                 pred_boxes = output["boxes"].cpu().numpy()
#                 pred_scores = output["scores"].cpu().numpy()
#                 pred_labels = output["labels"].cpu().numpy()
#                 for pb, ps, pl in zip(pred_boxes, pred_scores, pred_labels):
#                     if pl == 0:
#                         continue
#                     preds_by_class[pl].append((ps, idx, pb))
#     APs = {}
#     for cls in range(1, 21):
#         if npos[cls] == 0:
#             continue
#         preds_cls = sorted(preds_by_class[cls], key=lambda x: x[0], reverse=True)
#         tp = np.zeros(len(preds_cls), dtype=np.float32)
#         fp = np.zeros(len(preds_cls), dtype=np.float32)
#         matched = {img_id: np.zeros(len(gt_by_class[cls].get(img_id, [])), dtype=bool) for img_id in gt_by_class[cls]}
#         for i, (_, img_id, pred_box) in enumerate(preds_cls):
#             if img_id in gt_by_class[cls]:
#                 best_iou = 0.0
#                 best_gt_idx = -1
#                 for gt_idx, gt_box in enumerate(gt_by_class[cls][img_id]):
#                     if not matched[img_id][gt_idx]:
#                         gt_box = np.array(gt_box, dtype=np.float32)
#                         pred_box = pred_box.astype(np.float32)
#                         ixmin = max(gt_box[0], pred_box[0])
#                         iymin = max(gt_box[1], pred_box[1])
#                         ixmax = min(gt_box[2], pred_box[2])
#                         iymax = min(gt_box[3], pred_box[3])
#                         iw = max(ixmax - ixmin + 1.0, 0.0)
#                         ih = max(iymax - iymin + 1.0, 0.0)
#                         inter = iw * ih
#                         gt_area = (gt_box[2]-gt_box[0]+1.0)*(gt_box[3]-gt_box[1]+1.0)
#                         pred_area = (pred_box[2]-pred_box[0]+1.0)*(pred_box[3]-pred_box[1]+1.0)
#                         union = gt_area + pred_area - inter
#                         iou = inter / union if union > 0 else 0.0
#                         if iou > best_iou:
#                             best_iou = iou
#                             best_gt_idx = gt_idx
#                 if best_iou >= 0.5:
#                     tp[i] = 1
#                     matched[img_id][best_gt_idx] = True
#                 else:
#                     fp[i] = 1
#             else:
#                 fp[i] = 1
#         tp_cum = np.cumsum(tp)
#         fp_cum = np.cumsum(fp)
#         rec = tp_cum / npos[cls]
#         prec = tp_cum / (tp_cum + fp_cum + 1e-6)
#         mrec = np.concatenate(([0.0], rec, [1.0]))
#         mprec = np.concatenate(([0.0], prec, [0.0]))
#         for j in range(len(mprec)-2, -1, -1):
#             mprec[j] = max(mprec[j], mprec[j+1])
#         idxs = np.where(mrec[1:] != mrec[:-1])[0]
#         ap = 0.0
#         for j in idxs:
#             ap += (mrec[j+1] - mrec[j]) * mprec[j+1]
#         APs[cls] = ap
#     mAP = np.mean(list(APs.values())) if APs else 0.0
#     return mAP, APs

# def evaluate_segmentation(model, data_loader):
#     model.eval()
#     num_classes = 21  
#     total_inter = np.zeros(num_classes, dtype=np.int64)
#     total_union = np.zeros(num_classes, dtype=np.int64)
#     with torch.no_grad():
#         for images, masks in data_loader:
#             images = images.to(device)
#             masks = masks.to(device)
#             outputs = model(images)
#             preds = outputs['out']  
#             preds = preds.argmax(dim=1)  
#             for pred, mask in zip(preds, masks):
#                 pred = pred.cpu().numpy()
#                 mask = mask.cpu().numpy()
#                 valid = mask != 255 
#                 for cls in range(num_classes):
#                     curr_pred = (pred == cls) & valid
#                     curr_gt   = (mask == cls) & valid
#                     inter = np.logical_and(curr_pred, curr_gt).sum()
#                     union = np.logical_or(curr_pred, curr_gt).sum()
#                     total_inter[cls] += inter
#                     total_union[cls] += union
#     IoUs = []
#     for cls in range(1, num_classes): 
#         if total_union[cls] == 0:
#             IoU = 0.0
#         else:
#             IoU = total_inter[cls] / total_union[cls]
#         IoUs.append(IoU)
#     mIoU = np.mean(IoUs) if IoUs else 0.0
#     return mIoU, IoUs

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_det_baseline = model_det_baseline.to(device)
# model_det_moco     = model_det_moco.to(device)
# model_seg_baseline = model_seg_baseline.to(device)
# model_seg_moco     = model_seg_moco.to(device)


In [ ]:
# from tqdm import tqdm

# optimizer_det_baseline = torch.optim.SGD(model_det_baseline.parameters(), lr=0.005, momentum=0.9, weight_decay=1e-4)
# optimizer_det_moco     = torch.optim.SGD(model_det_moco.parameters(), lr=0.005, momentum=0.9, weight_decay=1e-4)
# scheduler_det_baseline = torch.optim.lr_scheduler.StepLR(optimizer_det_baseline, step_size=20, gamma=0.1)
# scheduler_det_moco     = torch.optim.lr_scheduler.StepLR(optimizer_det_moco, step_size=20, gamma=0.1)

# optimizer_seg_baseline = torch.optim.SGD(model_seg_baseline.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
# optimizer_seg_moco     = torch.optim.SGD(model_seg_moco.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
# scheduler_seg_baseline = torch.optim.lr_scheduler.StepLR(optimizer_seg_baseline, step_size=40, gamma=0.1)
# scheduler_seg_moco     = torch.optim.lr_scheduler.StepLR(optimizer_seg_moco, step_size=40, gamma=0.1)

# criterion_seg = torch.nn.CrossEntropyLoss(ignore_index=255)

# num_epochs_det = 5
# num_epochs_seg = 5

# mAP_baseline_history = []
# mAP_moco_history = []
# mIoU_baseline_history = []
# mIoU_moco_history = []

# print("Starting training...")
# for epoch in range(1, num_epochs_det+1):
#     model_det_baseline.train()
#     model_det_moco.train()
#     epoch_loss_baseline = 0.0
#     epoch_loss_moco = 0.0
#     for images, targets in tqdm(train_loader_det):
#         images = [img.to(device) for img in images]
#         targets_baseline = [{k: v.to(device) for k,v in tgt.items()} for tgt in targets]
#         targets_moco     = [{k: v.to(device) for k,v in tgt.items()} for tgt in targets]
#         loss_dict_base = model_det_baseline(images, targets_baseline)
#         losses_base = sum(loss for loss in loss_dict_base.values())
#         optimizer_det_baseline.zero_grad()
#         losses_base.backward()
#         optimizer_det_baseline.step()
#         epoch_loss_baseline += losses_base.item()
#         loss_dict_moco = model_det_moco(images, targets_moco)
#         losses_moco = sum(loss for loss in loss_dict_moco.values())
#         optimizer_det_moco.zero_grad()
#         losses_moco.backward()
#         optimizer_det_moco.step()
#         epoch_loss_moco += losses_moco.item()
#     scheduler_det_baseline.step()
#     scheduler_det_moco.step()
#     mAP_base, _ = evaluate_detection(model_det_baseline, test_loader_det)
#     mAP_moco, _ = evaluate_detection(model_det_moco, test_loader_det)
#     mAP_baseline_history.append(mAP_base)
#     mAP_moco_history.append(mAP_moco)
#     print(f"[Detection] Epoch {epoch}: Baseline mAP={mAP_base:.4f}, MoCo mAP={mAP_moco:.4f}")



In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(6,4))
# plt.plot(range(1, num_epochs_det+1), mAP_baseline_history, label='Baseline (Supervised)', marker='o')
# plt.plot(range(1, num_epochs_det+1), mAP_moco_history, label='MoCo v2 (Self-Supervised)', marker='s')
# plt.xlabel('Epoch')
# plt.ylabel('mAP (VOC 2007 Test)')
# plt.title('Object Detection mAP vs Epoch')
# plt.legend()
# plt.grid(True)
# plt.tight_layout()
# plt.savefig('detection_mAP_curve.png')
# plt.show()






In [ ]:
# for epoch in range(1, num_epochs_seg+1):
#     model_seg_baseline.train()
#     model_seg_moco.train()
#     epoch_loss_base = 0.0
#     epoch_loss_moco = 0.0
#     for images, masks in tqdm(train_loader_seg):
#         images = images.to(device)
#         masks  = masks.to(device)
#         outputs_base = model_seg_baseline(images)
#         main_out_base = outputs_base['out']
#         aux_out_base  = outputs_base['aux'] if 'aux' in outputs_base else None
#         loss_main_base = criterion_seg(main_out_base, masks)
#         loss_aux_base = criterion_seg(aux_out_base, masks) if aux_out_base is not None else 0
#         loss_total_base = loss_main_base + 0.4 * loss_aux_base 
#         optimizer_seg_baseline.zero_grad()
#         loss_total_base.backward()
#         optimizer_seg_baseline.step()
#         epoch_loss_base += loss_total_base.item()
#         outputs_moco = model_seg_moco(images)
#         main_out_moco = outputs_moco['out']
#         aux_out_moco  = outputs_moco['aux'] if 'aux' in outputs_moco else None
#         loss_main_moco = criterion_seg(main_out_moco, masks)
#         loss_aux_moco = criterion_seg(aux_out_moco, masks) if aux_out_moco is not None else 0
#         loss_total_moco = loss_main_moco + 0.4 * loss_aux_moco
#         optimizer_seg_moco.zero_grad()
#         loss_total_moco.backward()
#         optimizer_seg_moco.step()
#         epoch_loss_moco += loss_total_moco.item()
#     scheduler_seg_baseline.step()
#     scheduler_seg_moco.step()
#     mIoU_base, _ = evaluate_segmentation(model_seg_baseline, val_loader_seg)
#     mIoU_moco, _ = evaluate_segmentation(model_seg_moco, val_loader_seg)
#     mIoU_baseline_history.append(mIoU_base)
#     mIoU_moco_history.append(mIoU_moco)
#     print(f"[Segmentation] Epoch {epoch}: Baseline mIoU={mIoU_base:.4f}, MoCo mIoU={mIoU_moco:.4f}")
# print("Training completed.")


In [ ]:
# plt.figure(figsize=(6,4))
# plt.plot(range(1, num_epochs_seg+1), mIoU_baseline_history, label='Baseline (Supervised)', marker='o')
# plt.plot(range(1, num_epochs_seg+1), mIoU_moco_history, label='MoCo v2 (Self-Supervised)', marker='s')
# plt.xlabel('Epoch')
# plt.ylabel('mIoU (VOC 2012 Val)')
# plt.title('Segmentation mIoU vs Epoch')
# plt.legend()
# plt.grid(True)
# plt.tight_layout()
# plt.savefig('segmentation_mIoU_curve.png')
# plt.show()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models, datasets
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.backbone_utils import BackboneWithFPN
from torchvision.ops import FeaturePyramidNetwork
from torchvision.models.detection.backbone_utils import LastLevelMaxPool
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from tqdm import tqdm


In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

VOC_CLASSES = [
    'aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
    'bus', 'car', 'cat', 'chair', 'cow',
    'diningtable', 'dog', 'horse', 'motorbike', 'person',
    'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
]
CLASS_TO_IDX = {cls_name: i+1 for i, cls_name in enumerate(VOC_CLASSES)}

MEAN = [0.485, 0.456, 0.406]
STD  = [0.229, 0.224, 0.225]

class VOCDetectionDataset(datasets.VOCDetection):
    def __init__(self, root, year="2007", image_set="train"):
        super().__init__(root=root, year=year, image_set=image_set, download=False)

    def __getitem__(self, index):
        img, target = super().__getitem__(index)

        ann = target["annotation"]
        width = int(ann["size"]["width"])
        height = int(ann["size"]["height"])

        objects = ann.get("object", [])
        if not isinstance(objects, list):
            objects = [objects]

        boxes = []
        labels = []

        for obj in objects:
            if int(obj.get("difficult", 0)) == 1:
                continue

            name = obj["name"]
            if name not in CLASS_TO_IDX:
                continue

            bbox = obj["bndbox"]
            xmin = float(bbox["xmin"]) - 1
            ymin = float(bbox["ymin"]) - 1
            xmax = float(bbox["xmax"]) - 1
            ymax = float(bbox["ymax"]) - 1

            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(CLASS_TO_IDX[name])

        if len(boxes) == 0:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)
        else:
            boxes = torch.tensor(boxes, dtype=torch.float32)
            labels = torch.tensor(labels, dtype=torch.int64)

        img = transforms.functional.to_tensor(img)
        img = transforms.functional.normalize(img, MEAN, STD)

        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": torch.tensor([index])
        }

        return img, target

class VOCSegmentationDataset(datasets.VOCSegmentation):
    def __init__(self, root, year="2007", image_set="train", resize=None):
        super().__init__(root=root, year=year, image_set=image_set, download=False)
        self.resize = resize

    def __getitem__(self, index):
        img, mask = super().__getitem__(index)

        if self.resize:
            img = img.resize(self.resize, Image.BILINEAR)
            mask = mask.resize(self.resize, Image.NEAREST)

        img = transforms.functional.to_tensor(img)
        img = transforms.functional.normalize(img, MEAN, STD)

        mask = torch.as_tensor(np.array(mask), dtype=torch.long)

        return img, mask


VOC_ROOT_TRAIN = "/kaggle/input/pascal-voc-2007/VOCtrainval_06-Nov-2007"
VOC_ROOT_TEST  = "/kaggle/input/pascal-voc-2007/VOCtest_06-Nov-2007"

transform_det = transforms.ToTensor()  
resize_size = (512, 512)  
transform_seg_img = transforms.Normalize(mean=MEAN, std=STD)

# Датасеты
train_det_dataset = VOCDetectionDataset(
    root=VOC_ROOT_TRAIN,
    year="2007",
    image_set="trainval"
)

test_det_dataset = VOCDetectionDataset(
    root=VOC_ROOT_TEST,
    year="2007",
    image_set="test"
)



train_seg_dataset = VOCSegmentationDataset(
    root=VOC_ROOT_TRAIN,
    year="2007",
    image_set="trainval",
    resize=(512, 512)
)

test_seg_dataset = VOCSegmentationDataset(
    root=VOC_ROOT_TEST,
    year="2007",
    image_set="test",
    resize=(512, 512)
)


In [ ]:


train_det_loader = torch.utils.data.DataLoader(train_det_dataset, batch_size=2, shuffle=True, 
                                              collate_fn=lambda batch: tuple(zip(*batch)))
test_det_loader  = torch.utils.data.DataLoader(test_det_dataset, batch_size=2, shuffle=False, 
                                              collate_fn=lambda batch: tuple(zip(*batch)))
train_seg_loader = torch.utils.data.DataLoader(train_seg_dataset, batch_size=4, shuffle=True)
test_seg_loader  = torch.utils.data.DataLoader(test_seg_dataset, batch_size=4, shuffle=False)

def load_moco_weights(weights_path):
    checkpoint = torch.load(weights_path, map_location='cpu')
    state_dict = checkpoint.get('state_dict', checkpoint)
    new_state_dict = {}
    for k, v in state_dict.items():
        if k.startswith('module.encoder_q.'):
            new_key = k[len('module.encoder_q.'):]
        elif k.startswith('encoder_q.'):
            new_key = k[len('encoder_q.'):]
        else:
            new_key = k
        if new_key.startswith('fc.') or new_key.startswith('module.fc.') or new_key.startswith('encoder_q.fc.'):
            continue
        new_state_dict[new_key] = v
    return new_state_dict

moco_weights_path = "/kaggle/input/pretrained-weights-mocov2/moco_v2_800ep_pretrain.pth.tar"
moco_state = load_moco_weights(moco_weights_path)

resnet50_imagenet = models.resnet50(pretrained=True)
resnet50_moco = models.resnet50(pretrained=False)
resnet50_moco.load_state_dict(moco_state, strict=False)


backbone_base = resnet_fpn_backbone(
    backbone_name="resnet50",
    pretrained=True,         
    trainable_layers=5
)
backbone_moco = resnet_fpn_backbone(
    backbone_name="resnet50",
    pretrained=False,         
    trainable_layers=5
)
num_classes = 21 
model_det_base = FasterRCNN(backbone_base, num_classes=21)
model_det_moco = FasterRCNN(backbone_moco, num_classes=num_classes)

model_seg_base = models.segmentation.deeplabv3_resnet50(pretrained=False, num_classes=len(VOC_CLASSES)+1)
model_seg_moco = models.segmentation.deeplabv3_resnet50(pretrained=False, num_classes=len(VOC_CLASSES)+1)
model_seg_base.backbone.load_state_dict(resnet50_imagenet.state_dict(), strict=False)
model_seg_moco.backbone.load_state_dict(resnet50_moco.state_dict(), strict=False)
if hasattr(model_seg_base, 'aux_classifier'):
    model_seg_base.aux_classifier = None
    model_seg_moco.aux_classifier = None


model_det_base.to(device)
model_det_moco.to(device)
model_seg_base.to(device)
model_seg_moco.to(device)

optimizer_det_base = optim.SGD(model_det_base.parameters(), lr=0.005, momentum=0.9, weight_decay=1e-4)
optimizer_det_moco = optim.SGD(model_det_moco.parameters(), lr=0.005, momentum=0.9, weight_decay=1e-4)
optimizer_seg_base = optim.SGD(model_seg_base.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
optimizer_seg_moco = optim.SGD(model_seg_moco.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
scheduler_seg_base = optim.lr_scheduler.StepLR(optimizer_seg_base, step_size=20, gamma=0.1)
scheduler_seg_moco = optim.lr_scheduler.StepLR(optimizer_seg_moco, step_size=20, gamma=0.1)

criterion_seg = nn.CrossEntropyLoss(ignore_index=255)  

In [ ]:
def evaluate_detection(model, loader):
    model.eval()
    gt_boxes_per_class = {cls_idx: [] for cls_idx in range(1, len(VOC_CLASSES)+1)}
    pred_data_per_class = {cls_idx: [] for cls_idx in range(1, len(VOC_CLASSES)+1)}
    for images, targets in tqdm(loader):
        for target in targets:
            labels = target['labels'].tolist()
            boxes = target['boxes'].tolist()
            for lbl, box in zip(labels, boxes):
                if lbl == 0:
                    continue
                gt_boxes_per_class[lbl].append({"box": box, "used": False})
    with torch.no_grad():
        for images, targets in loader:
            images = [img.to(device) for img in images]
            outputs = model(images)
            for output, target in zip(outputs, targets):
                pred_boxes = output['boxes'].tolist()
                pred_scores = output['scores'].tolist()
                pred_labels = output['labels'].tolist()
                for box, score, lbl in zip(pred_boxes, pred_scores, pred_labels):
                    if lbl == 0:
                        continue
                    pred_data_per_class[lbl].append({"score": score, "box": box, "image_id": target['image_id'].item()})
    aps = []
    for cls_idx in range(1, len(VOC_CLASSES)+1):
        preds = pred_data_per_class[cls_idx]
        if len(gt_boxes_per_class[cls_idx]) == 0:
            continue
        if len(preds) == 0:
            aps.append(0.0)
            continue
        preds.sort(key=lambda x: x['score'], reverse=True)
        tp = []
        fp = []
        gt_for_class = gt_boxes_per_class[cls_idx]
        total_gt = len(gt_for_class)
        for pred in preds:
            best_iou = 0.0
            best_gt_idx = -1
            for gt_idx, gt in enumerate(gt_for_class):
                if gt['used']:
                    continue
               
                box_pred = pred['box']
                box_gt = gt['box']
                inter_xmin = max(box_pred[0], box_gt[0])
                inter_ymin = max(box_pred[1], box_gt[1])
                inter_xmax = min(box_pred[2], box_gt[2])
                inter_ymax = min(box_pred[3], box_gt[3])
                if inter_xmax < inter_xmin or inter_ymax < inter_ymin:
                    iou = 0.0
                else:
                    inter_area = (inter_xmax - inter_xmin) * (inter_ymax - inter_ymin)
                    pred_area = (box_pred[2] - box_pred[0]) * (box_pred[3] - box_pred[1])
                    gt_area = (box_gt[2] - box_gt[0]) * (box_gt[3] - box_gt[1])
                    union_area = pred_area + gt_area - inter_area
                    iou = inter_area / union_area if union_area > 0 else 0.0
                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = gt_idx
            if best_iou >= 0.5 and best_gt_idx >= 0:
                if not gt_for_class[best_gt_idx]['used']:
                    tp.append(1)
                    fp.append(0)
                    gt_for_class[best_gt_idx]['used'] = True 
                else:
                    tp.append(0)
                    fp.append(1)
            else:
                tp.append(0)
                fp.append(1)
        tp = np.array(tp)
        fp = np.array(fp)
        cum_tp = np.cumsum(tp)
        cum_fp = np.cumsum(fp)
        recalls = cum_tp / float(total_gt)
        precisions = cum_tp / np.maximum(cum_tp + cum_fp, 1e-6)
        recalls = np.concatenate(([0.0], recalls, [1.0]))
        precisions = np.concatenate(([1.0], precisions, [0.0]))
        for i in range(len(precisions)-2, -1, -1):
            precisions[i] = max(precisions[i], precisions[i+1])
        indices = np.where(recalls[1:] != recalls[:-1])[0]
        ap = 0.0
        for i in indices:
            ap += (recalls[i+1] - recalls[i]) * precisions[i+1]
        aps.append(ap)
    mAP = 100 * np.mean(aps) if aps else 0.0
    return mAP


In [15]:



def evaluate_segmentation(model, loader):
    model.eval()
    num_classes = len(VOC_CLASSES) + 1 
    total_intersect = np.zeros(num_classes, dtype=np.int64)
    total_union = np.zeros(num_classes, dtype=np.int64)
    with torch.no_grad():
        for images, masks in tqdm(loader):
            images = images.to(device)
            masks = masks.numpy()  #
            outputs = model(images)['out']  
            preds = outputs.argmax(dim=1).cpu().numpy()  
            for pred_mask, true_mask in zip(preds, masks):
                mask_ignore = (true_mask == 255)
                for cls_idx in range(num_classes):
                    pred_i = (pred_mask == cls_idx) & (~mask_ignore)
                    true_i = (true_mask == cls_idx) & (~mask_ignore)
                    inter = np.logical_and(pred_i, true_i).sum()
                    union = np.logical_or(pred_i, true_i).sum()
                    total_intersect[cls_idx] += inter
                    total_union[cls_idx] += union
    ious = []
    for cls_idx in range(num_classes):
        if total_union[cls_idx] == 0:
            continue
        iou = total_intersect[cls_idx] / float(total_union[cls_idx])
        ious.append(iou)
    mIoU = 100 * np.mean(ious) if ious else 0.0
    return mIoU

def train_det_epoch(model, optimizer, loader):
    model.train()
    total_loss = 0.0
    for images, targets in tqdm(loader):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in tgt.items()} for tgt in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        total_loss += losses.item()
    return total_loss / len(loader)

def train_seg_epoch(model, optimizer, loader, criterion):
    model.train()
    total_loss = 0.0
    for images, masks in tqdm(loader):
        images = images.to(device)
        masks = masks.to(device)
        outputs = model(images)['out']  
        loss = criterion(outputs, masks)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


Начало обучения...

=== Обучение детекции (ResNet50 MoCo v2) ===


  1%|          | 20/2506 [00:12<24:53,  1.66it/s]


KeyboardInterrupt: 

In [ ]:

det_epochs = 4
seg_epochs = 25

det_base_map_history = []
det_moco_map_history = []
seg_base_miou_history = []
seg_moco_miou_history = []

print("Начало обучения...")

print("\n=== Обучение детекции (ResNet50 ImageNet) ===")
for epoch in range(1, det_epochs+1):
    avg_loss = train_det_epoch(model_det_base, optimizer_det_base, train_det_loader)
    mAP = evaluate_detection(model_det_base, test_det_loader)
    det_base_map_history.append(mAP)
    print(f"Epoch {epoch}/{det_epochs} - Loss: {avg_loss:.3f}, Val mAP: {mAP:.2f}%")

print("\n=== Обучение детекции (ResNet50 MoCo v2) ===")
for epoch in range(1, det_epochs+1):
    avg_loss = train_det_epoch(model_det_moco, optimizer_det_moco, train_det_loader)
    mAP = evaluate_detection(model_det_moco, test_det_loader)
    det_moco_map_history.append(mAP)
    print(f"Epoch {epoch}/{det_epochs} - Loss: {avg_loss:.3f}, Val mAP: {mAP:.2f}%")

print("\n=== Обучение сегментации (ResNet50 ImageNet) ===")
for epoch in range(1, seg_epochs+1):
    avg_loss = train_seg_epoch(model_seg_base, optimizer_seg_base, train_seg_loader, criterion_seg)
    mIoU = evaluate_segmentation(model_seg_base, test_seg_loader)
    seg_base_miou_history.append(mIoU)
    print(f"Epoch {epoch}/{seg_epochs} - Loss: {avg_loss:.3f}, Val mIoU: {mIoU:.2f}%")
    scheduler_seg_base.step()

print("\n=== Обучение сегментации (ResNet50 MoCo v2) ===")
for epoch in range(1, seg_epochs+1):
    avg_loss = train_seg_epoch(model_seg_moco, optimizer_seg_moco, train_seg_loader, criterion_seg)
    mIoU = evaluate_segmentation(model_seg_moco, test_seg_loader)
    seg_moco_miou_history.append(mIoU)
    print(f"Epoch {epoch}/{seg_epochs} - Loss: {avg_loss:.3f}, Val mIoU: {mIoU:.2f}%")
    scheduler_seg_moco.step()

# Сохранение моделей
torch.save(model_det_base.state_dict(), "fasterrcnn_resnet50_baseline.pth")
torch.save(model_det_moco.state_dict(), "fasterrcnn_resnet50_moco.pth")
torch.save(model_seg_base.state_dict(), "deeplabv3_resnet50_baseline.pth")
torch.save(model_seg_moco.state_dict(), "deeplabv3_resnet50_moco.pth")

plt.figure()
epochs_det = range(1, det_epochs+1)
plt.plot(epochs_det, det_base_map_history, 'o-', label="Det mAP (ImageNet)")
plt.plot(epochs_det, det_moco_map_history, 'o-', label="Det mAP (MoCo v2)")
plt.title("Detection mAP per Epoch (Pascal VOC 2007)")
plt.xlabel("Epoch")
plt.ylabel("mAP (%)")
plt.legend()
plt.grid(True)
plt.savefig("detection_mAP.png")
plt.show()

plt.figure()
epochs_seg = range(1, seg_epochs+1)
plt.plot(epochs_seg, seg_base_miou_history, 'o-', label="Seg mIoU (ImageNet)")
plt.plot(epochs_seg, seg_moco_miou_history, 'o-', label="Seg mIoU (MoCo v2)")
plt.title("Segmentation mIoU per Epoch (Pascal VOC 2007)")
plt.xlabel("Epoch")
plt.ylabel("mIoU (%)")
plt.legend()
plt.grid(True)
plt.savefig("segmentation_mIoU.png")
plt.show()
print("\nОбучение завершено. Модели и графики сохранены.")